In [20]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)


def preprocess_data(targets, observed, estimated, test):
    """
    Preprocess the data by resampling, merging with targets, and dropping unnecessary columns.
    
    Parameters:
    - targets: Target dataframe with 'time' and target values.
    - observed: Dataframe with observed features.
    - estimated: Dataframe with estimated features.
    - test: Dataframe with test features.
    
    Returns:
    - Preprocessed dataframes ready for training and testing.
    """

    # Ensure the datetime columns are in datetime format
    targets['time'] = pd.to_datetime(targets['time'])
    observed['date_forecast'] = pd.to_datetime(observed['date_forecast'])
    estimated['date_forecast'] = pd.to_datetime(estimated['date_forecast'])
    test['date_forecast'] = pd.to_datetime(test['date_forecast'])

    # Ensure data is sorted by date_forecast
    targets = targets.sort_values(by='time')
    observed = observed.sort_values(by='date_forecast')
    estimated = estimated.sort_values(by='date_forecast')
    test = test.sort_values(by='date_forecast')

    # Identify boolean columns
    boolean_features = [col for col in observed.columns if observed[col].dropna().isin([0.0, 1.0]).all()]

    # Forward fill NaNs for boolean columns
    for df in [observed, estimated, test]:
        df[boolean_features] = df[boolean_features].fillna(method='ffill')

    # Forward fill for time-series data (for non-boolean columns)
    for df in [observed, estimated, test]:
        df[df.columns.difference(boolean_features)] = df[df.columns.difference(boolean_features)].fillna(method='ffill')

    """  
    # Forward fill for time-series data
    observed.fillna(method='ffill', inplace=True)
    estimated.fillna(method='ffill', inplace=True)
    test.fillna(method='ffill', inplace=True)

    # Fill NaNs in boolean features with 0
    boolean_features = [col for col in observed.columns if observed[col].dropna().isin([0.0, 1.0]).all()]
    observed[boolean_features] = observed[boolean_features].fillna(method='ffill')
    estimated[boolean_features] = estimated[boolean_features].fillna(method='ffill')
    test[boolean_features] = test[boolean_features].fillna(method='ffill') 
    """

    # Resample observed, estimated, and test data to 1 hour using mean() as aggregator
    # and drop rows where all columns are NaN
    observed_resampled = observed.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()
    estimated_resampled = estimated.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()
    test_resampled = test.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()

    # Round boolean columns after resampling
    for df in [observed_resampled, estimated_resampled, test_resampled]:
        df[boolean_features] = df[boolean_features].round(0)

    observed_resampled['estimated'] = 0
    estimated_resampled['estimated'] = 1
    test_resampled['estimated'] = 1
    
    # Merge the observed and estimated data
    weather_data = pd.concat([observed_resampled, estimated_resampled])

    # Merge with target values
    merged_data = pd.merge(targets, weather_data, how='inner', left_on='time', right_on='date_forecast')

    # Time-Based Features (training data)
    merged_data['hour'] = merged_data['date_forecast'].dt.hour
    merged_data['sin_hour'] = np.sin(2 * np.pi * merged_data['hour'] / 24)
    merged_data['cos_hour'] = np.cos(2 * np.pi * merged_data['hour'] / 24)
    # merged_data['day_of_week'] = merged_data['date_forecast'].dt.dayofweek
    merged_data['month'] = merged_data['date_forecast'].dt.month
    merged_data['sin_month'] = np.sin(2 * np.pi * merged_data['month'] / 12)
    merged_data['cos_month'] = np.cos(2 * np.pi * merged_data['month'] / 12)

    # Time-Based Features (test data)
    test_resampled['hour'] = test_resampled['date_forecast'].dt.hour
    test_resampled['sin_hour'] = np.sin(2 * np.pi * test_resampled['hour'] / 24)
    test_resampled['cos_hour'] = np.cos(2 * np.pi * test_resampled['hour'] / 24)
    # test_resampled['day_of_week'] = test_resampled['date_forecast'].dt.dayofweek
    test_resampled['month'] = test_resampled['date_forecast'].dt.month
    test_resampled['sin_month'] = np.sin(2 * np.pi * test_resampled['month'] / 12)
    test_resampled['cos_month'] = np.cos(2 * np.pi * test_resampled['month'] / 12)


    # Drop non-feature columns
    merged_data = merged_data.drop(columns=['time', 'date_forecast', 'pv_measurement', 'snow_density:kgm3'])
    test_resampled = test_resampled.drop(columns=['date_forecast', 'snow_density:kgm3'])

    # fixing ceiling_height NaN value
    merged_data['ceiling_height_agl:m'].fillna(0, inplace=True)
    test_resampled['ceiling_height_agl:m'].fillna(0, inplace=True)
  
    
    return merged_data, test_resampled

locations = ['A', 'B', 'C']
location_mapping = {'A': 1, 'B': 2, 'C': 3}
all_predictions = []
all_predictions_rf = []


for loc in locations:
    # Load your data
    train = pd.read_parquet(f'{loc}/train_targets.parquet').fillna(0)
    X_train_estimated = pd.read_parquet(f'{loc}/X_train_estimated.parquet')
    X_train_observed = pd.read_parquet(f'{loc}/X_train_observed.parquet')
    X_test_estimated = pd.read_parquet(f'{loc}/X_test_estimated.parquet')

    # save as csv for analysis
    """ train.to_csv(f'{loc}_csv/train_targets.csv')
    X_train_estimated.to_csv(f'{loc}_csv/X_train_estimated.csv')
    X_train_observed.to_csv(f'{loc}_csv/X_train_observed.csv')
    X_test_estimated.to_csv(f'{loc}_csv/X_test_estimated.csv') """

   # Preprocess data
    X_train, X_test = preprocess_data(train, X_train_observed, X_train_estimated, X_test_estimated)
    
    X_train['location'] = location_mapping[loc]
    X_test['location'] = location_mapping[loc]

    X_train.to_csv('X_train.csv')
    X_test.to_csv('X_test.csv')

    y = train['pv_measurement'].values


    # Ensure X and y have the same length
    min_length = min(len(X_train), len(y))
    X_train, y_train = X_train.iloc[:min_length], y[:min_length]

    X_train_data, X_eval_data, y_train_data, y_eval_data = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
    
    # Create catboost Pool objects
    train_pool = Pool(data=X_train_data, label=y_train_data, cat_features=['location', 'estimated'])
    eval_pool = Pool(data=X_eval_data, label=y_eval_data, cat_features=['location', 'estimated'])

    # Initialize and Train model
    #categorical_features = ['dew_or_rime:idx', 'elevation:m', 'is_day:idx', 'is_in_shadow:idx', 'snow_drift:idx', 'wind_speed_w_1000hPa:ms']

    model = CatBoostRegressor(depth=9, iterations=1000, loss_function='MAE')
    model.fit(train_pool, use_best_model=True, eval_set=eval_pool)

    # Make predictions using X_test_estimated data
    predictions = model.predict(X_test)
    
    # Store the predictions in all_predictions list
    all_predictions.append(predictions)

    """ 
    # Initialize and Train RandomForest model
    model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
    model_rf.fit(X_train, y_train)

    # Make predictions using X_test data
    predictions_rf = model_rf.predict(X_test)
    
    # Store the RandomForest predictions in all_predictions_rf list
    all_predictions_rf.append(predictions_rf) 
    
    final_predictions_rf = np.concatenate(all_predictions_rf)
    
    average_predictions = (np.array(final_predictions) + np.array(final_predictions_rf)) / 2.0
    """

# Concatenate all predictions
final_predictions = np.concatenate(all_predictions)


# Save the final_predictions to CSV
df = pd.DataFrame(final_predictions, columns=['prediction'])
df['id'] = df.index
df = df[['id', 'prediction']]
df['prediction'] = df['prediction'].apply(lambda x: max(0, x))
df.to_csv('final_predictions.csv', index=False)


/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_72994/1581591451.py:66: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()
/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_72994/1581591451.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_resampled = test.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()


0:	learn: 620.5536740	test: 604.2824538	best: 604.2824538 (0)	total: 83.3ms	remaining: 1m 23s
1:	learn: 605.0698998	test: 588.9310427	best: 588.9310427 (1)	total: 100ms	remaining: 50.1s
2:	learn: 592.0269717	test: 576.2088076	best: 576.2088076 (2)	total: 116ms	remaining: 38.7s
3:	learn: 582.8648192	test: 567.3252920	best: 567.3252920 (3)	total: 132ms	remaining: 32.9s
4:	learn: 570.8352827	test: 555.4953762	best: 555.4953762 (4)	total: 148ms	remaining: 29.5s
5:	learn: 557.3269148	test: 542.2484475	best: 542.2484475 (5)	total: 164ms	remaining: 27.1s
6:	learn: 544.9186969	test: 530.0566585	best: 530.0566585 (6)	total: 179ms	remaining: 25.4s
7:	learn: 533.6498660	test: 518.9417877	best: 518.9417877 (7)	total: 196ms	remaining: 24.3s
8:	learn: 520.8704248	test: 506.7372671	best: 506.7372671 (8)	total: 211ms	remaining: 23.2s
9:	learn: 508.2815992	test: 494.5485631	best: 494.5485631 (9)	total: 227ms	remaining: 22.5s
10:	learn: 498.3567323	test: 484.9775523	best: 484.9775523 (10)	total: 243ms	r

/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_72994/1581591451.py:66: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()
/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_72994/1581591451.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_resampled = test.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()


0:	learn: 95.5664717	test: 93.6988597	best: 93.6988597 (0)	total: 27ms	remaining: 26.9s
1:	learn: 93.8125802	test: 91.9781963	best: 91.9781963 (1)	total: 43.2ms	remaining: 21.6s
2:	learn: 92.3200315	test: 90.5332988	best: 90.5332988 (2)	total: 57.8ms	remaining: 19.2s
3:	learn: 90.5013489	test: 88.7437473	best: 88.7437473 (3)	total: 74.9ms	remaining: 18.6s
4:	learn: 88.8027720	test: 87.0458850	best: 87.0458850 (4)	total: 90.4ms	remaining: 18s
5:	learn: 87.0380122	test: 85.3342021	best: 85.3342021 (5)	total: 106ms	remaining: 17.5s
6:	learn: 85.4513344	test: 83.7831843	best: 83.7831843 (6)	total: 122ms	remaining: 17.2s
7:	learn: 83.8860136	test: 82.2479230	best: 82.2479230 (7)	total: 141ms	remaining: 17.5s
8:	learn: 82.4850879	test: 80.8685382	best: 80.8685382 (8)	total: 214ms	remaining: 23.6s
9:	learn: 80.8064528	test: 79.2291493	best: 79.2291493 (9)	total: 235ms	remaining: 23.3s
10:	learn: 79.2768080	test: 77.7319198	best: 77.7319198 (10)	total: 254ms	remaining: 22.8s
11:	learn: 77.6420

/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_72994/1581591451.py:66: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  estimated_resampled = estimated.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()
/var/folders/f6/kxzcl1nj68bdr_rt5h1lbr180000gn/T/ipykernel_72994/1581591451.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  test_resampled = test.set_index('date_forecast').resample('1H').mean().dropna(how='all').reset_index()


0:	learn: 62.2102017	test: 60.6318345	best: 60.6318345 (0)	total: 30.7ms	remaining: 30.7s
1:	learn: 61.5255057	test: 59.9359831	best: 59.9359831 (1)	total: 50.2ms	remaining: 25s
2:	learn: 60.5477061	test: 58.9687481	best: 58.9687481 (2)	total: 68.6ms	remaining: 22.8s
3:	learn: 59.4372395	test: 57.8794698	best: 57.8794698 (3)	total: 88.3ms	remaining: 22s
4:	learn: 58.5283735	test: 56.9601708	best: 56.9601708 (4)	total: 108ms	remaining: 21.4s
5:	learn: 57.6574377	test: 56.0856754	best: 56.0856754 (5)	total: 125ms	remaining: 20.7s
6:	learn: 56.7222082	test: 55.1706793	best: 55.1706793 (6)	total: 147ms	remaining: 20.9s
7:	learn: 55.8611569	test: 54.3205254	best: 54.3205254 (7)	total: 168ms	remaining: 20.8s
8:	learn: 54.9946825	test: 53.4698357	best: 53.4698357 (8)	total: 191ms	remaining: 21.1s
9:	learn: 54.1538813	test: 52.6693659	best: 52.6693659 (9)	total: 208ms	remaining: 20.6s
10:	learn: 53.3545323	test: 51.8799938	best: 51.8799938 (10)	total: 225ms	remaining: 20.2s
11:	learn: 52.49117